<a href="https://colab.research.google.com/github/bharatgirdhar/DeepVisionProject/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from tqdm import tqdm
import matplotlib.pyplot as plt

#from models import *
#from utils import progress_bar


#parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
#parser.add_argument('--resume', '-r', action='store_true',
#                    help='resume from checkpoint')
#args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')

#tMean = (0.4914, 0.4822, 0.4465)
#tStandardDev = (0.2023, 0.1994, 0.2010)
#ApplyTransformations(vMean,tStandardDev)



def CreateCIFAR10TrainSet(vBatchSize, transform_train):
  trainset = torchvision.datasets.CIFAR10(
      root='./data', train=True, download=True, transform=transform_train)
  trainloader = torch.utils.data.DataLoader(
      trainset, batch_size=vBatchSize, shuffle=True, num_workers=2)
  return trainloader
def CreateCIFAR10TestSet(vBatchSize, transform_test):
  testset = torchvision.datasets.CIFAR10(
      root='./data', train=False, download=True, transform=transform_test)
  testloader = torch.utils.data.DataLoader(
      testset, batch_size=vBatchSize, shuffle=False, num_workers=2)
  return testloader
#classes = ('plane', 'car', 'bird', 'cat', 'deer',
#           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
#print('==> Building model..')

  # net = VGG('VGG19')
#  net = ResNet18()
  # net = PreActResNet18()
  # net = GoogLeNet()
  # net = DenseNet121()
  # net = ResNeXt29_2x64d()
  # net = MobileNet()
  # net = MobileNetV2()
  # net = DPN92()
  # net = ShuffleNetG2()
  # net = SENet18()
  # net = ShuffleNetV2(1)
  # net = EfficientNetB0()
  # net = RegNetX_200MF()
  #net = SimpleDLA()
#  net = net.to(device)
#  if device == 'cuda':
#      net = torch.nn.DataParallel(net)
#      cudnn.benchmark = True

#if args.resume:
    # Load checkpoint.
#    print('==> Resuming from checkpoint..')
#    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#    checkpoint = torch.load('./checkpoint/ckpt.pth')
#    net.load_state_dict(checkpoint['net'])
#    best_acc = checkpoint['acc']
#    start_epoch = checkpoint['epoch']

def SetOptimizer(vOptimizerType, vlr, net):
  if vOptimizerType==1:
    optimizer = optim.SGD(net.parameters(), lr=vlr,
                      momentum=0.9, weight_decay=5e-4)
  return optimizer
def SetScheduler(optimizer):
  #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
  #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 0.19,,,512 lr_range=(0.1, 1.)
  return scheduler

def SetLoss(vLossType):
  criterion = nn.CrossEntropyLoss()
  return criterion

test_losses=[]
train_losses=[]

test_accuracies=[]
train_accuracies=[]

def train(model, device, train_loader, optimizer, epoch, vNormalizationType, criterion):
    model.train()
    train_loss=0
    pbar = tqdm(train_loader, leave="false")
    correct=0
    processed=0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data['image'].float().to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #pred = output.argmax(dim=1).float()
        #print(output.requires_grad)
        #pred.requires_grad=True
        #print(pred.size())
        #if vNormalizationType in (0,1):          
        loss = criterion(output, target)
        #else:          
        #  loss = F.l1_loss(pred,target.float())
          #print(pred.size())
          #print('target',target.size())
          #loss.requres_grad= False
          #vL1=0
          #for p in Net.Net(vNormalizationType).parameters():
          #  vL1 += p.abs().sum()
          #loss= loss + vL1Factor*vL1        
        #print('loss',loss)
        pred = output.argmax(dim=1)
        train_loss += loss
        loss.backward()        
        optimizer.step()
        
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100.*correct/processed:0.2f}')
    
    train_losses.append(train_loss/len(train_loader.dataset))
    train_accuracies.append(100.*correct/processed)
    
    print('\n\nEPOCH : ',epoch)
    print(f"\nTrain Loss{train_loss/len(train_loader.dataset)}")


def test(model, device, test_loader,epoch,vNormalizationType):
    model.eval()
    test_loss = 0
    correct = 0
    missed_pred=[]
    j=0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data_copy=data
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            #print(batch_idx)
            if epoch==1 and batch_idx==99:
              #print('Hi')
              for i in range(len(target)):
                #print('Hello')
                if pred[i]!=target[i]:
                  plt.imshow(data_copy[i].permute(1,2,0))
                  plt.show()
                  #break

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    test_accuracies.append(100. * correct / len(test_loader.dataset))
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


# Training
#def train(epoch):
#    print('\nEpoch: %d' % epoch)
#    net.train()
#    train_loss = 0
#    correct = 0
#    total = 0
#    for batch_idx, (inputs, targets) in enumerate(trainloader):
#        inputs, targets = inputs.to(device), targets.to(device)
#        optimizer.zero_grad()
#        outputs = net(inputs)
#        loss = criterion(outputs, targets)
#        loss.backward()
#        optimizer.step()

#        train_loss += loss.item()
#        _, predicted = outputs.max(1)
#        total += targets.size(0)
#        correct += predicted.eq(targets).sum().item()

#        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


#def test(epoch):
#    global best_acc
#    net.eval()
#    test_loss = 0
#    correct = 0
#    total = 0
#    with torch.no_grad():
#        for batch_idx, (inputs, targets) in enumerate(testloader):
#            inputs, targets = inputs.to(device), targets.to(device)
#            outputs = net(inputs)
#            loss = criterion(outputs, targets)

#            test_loss += loss.item()
#            _, predicted = outputs.max(1)
#            total += targets.size(0)
#            correct += predicted.eq(targets).sum().item()

#            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
#    acc = 100.*correct/total
#    if acc > best_acc:
#        print('Saving..')
#        state = {
#            'net': net.state_dict(),
#            'acc': acc,
#            'epoch': epoch,
#        }
#        if not os.path.isdir('checkpoint'):
#            os.mkdir('checkpoint')
#        torch.save(state, './checkpoint/ckpt.pth')
#        best_acc = acc


#for epoch in range(start_epoch, start_epoch+200):
#    train(epoch)
#    test(epoch)
#    scheduler.step()

SyntaxError: ignored